In [ ]:
# ! pip install -q kaggle

In [ ]:
# ! mkdir ~/.kaggle

In [ ]:
# ! cp kaggle.json ~/.kaggle/

In [ ]:
# ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets list

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tolgadincer/labeled-chest-xray-images")

print("Path to dataset files:", path)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from sklearn.model_selection import train_test_split
from pathlib import Path
import cv2
import random
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,Flatten,Dropout,BatchNormalization,SeparableConv2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam,RMSprop,Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,array_to_img,load_img
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.regularizers import l2

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
import numpy as np

In [ ]:
from tensorflow.keras.applications import VGG19

In [ ]:
Input_ = Path("/kaggle/input/chest-xray-images-guangzhou-women-and-childrens/chest_xray/chest_xray_ternary")

train_path = Input_/"train"

test_path = Input_/"test"

In [ ]:
Normal_train_dir = train_path/"NORMAL"
Bacterial_train_dir = train_path/"BACTERIAL"
Viral_train_dir = train_path/"VIRAL"

In [ ]:
Normal_train_cases = Normal_train_dir.glob("*.jpeg")
Bacterial_train_cases = Bacterial_train_dir.glob("*.jpeg")
Viral_train_cases = Viral_train_dir.glob("*.jpeg")

In [ ]:
data = []

for case in Normal_train_cases:
    data.append((case, 0))

i = 0 
for case in Bacterial_train_cases:
    data.append((case, 1))
    i = i + 1
    if i >= 1300:
        break
    
for case in Viral_train_cases:
    data.append((case,2))


data = pd.DataFrame(data,columns = ['image','label'],index=None)
data = data.sample(frac=1.).reset_index(drop=True)

X = data['image']
y = data['label']

In [ ]:
data.shape

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,random_state=12)

In [ ]:
X_train.shape,y_train.shape,X_val.shape,y_val.shape

In [ ]:
train_df = pd.DataFrame({'image':X_train,'label':y_train})
train_df['label'].value_counts()


In [ ]:
normal_cases_count = train_df[train_df['label'] == 0].shape[0]
bacteria_cases_count = train_df[train_df['label'] == 1].shape[0]
virus_cases_count = train_df[train_df['label'] == 2].shape[0]

In [ ]:
plt.bar(["Normal","bacteria","virus"],[normal_cases_count,bacteria_cases_count,virus_cases_count],color = ["green","red","blue"])

In [ ]:
Bacterial_Samples = (train_df[train_df["label"] == 1]['image'].iloc[:5]).tolist()
Normal_Samples = (train_df[train_df["label"] == 0]['image'].iloc[:5]).tolist()
Virus_Samples = (train_df[train_df["label"] == 0]['image'].iloc[:5]).tolist()


samples = Normal_Samples+ Bacterial_Samples +  Virus_Samples
del Bacterial_Samples,Normal_Samples,Virus_Samples

f,ax = plt.subplots(3,5,figsize=(30,15))

for i in range(15):
  img = cv2.imread(samples[i])
  ax[i//5,i%5].imshow(img)
  if(i < 5) :
    ax[i//5,i%5].set_title("Normal")
  elif(i < 10):
    ax[i//5,i%5].set_title("Bacterial")
  else:
      ax[i//5,i%5].set_title("Viral")
  ax[i//5,i%5].axis('off')
  ax[i//5,i%5].set_aspect('auto')
plt.show()

In [ ]:
valid_data = []
valid_labels = []
for img in X_val:
  img = cv2.imread(str(img))
  img = cv2.resize(img, (224,224))
  if img.shape[2] ==1:
    img = np.dstack([img, img, img])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = img.astype(np.float32)/255.
  valid_data.append(img)
for label in y_val:
  valid_labels.append(label)

valid_data = np.array(valid_data)
valid_labels = np.array(valid_labels)

In [ ]:
valid_labels = to_categorical(valid_labels,num_classes = 3)

In [ ]:
len(valid_data),len(valid_labels)

In [ ]:
train_data = []
train_labels = []
for img in X_train:
  img = cv2.imread(str(img))
  img = cv2.resize(img, (224,224))
  if img.shape[2] ==1:
    img = np.dstack([img, img, img])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = img.astype(np.float32)/255.
  train_data.append(img)

for label in y_train:
  train_labels.append(label)

train_data = np.array(train_data)
train_labels = np.array(train_labels)

In [ ]:
train_labels = to_categorical(train_labels,num_classes = 3)

In [ ]:
# datagen = ImageDataGenerator(
#     rotation_range = 10,
#     width_shift_range = 0.1,
#     height_shift_range = 0.1,
#     shear_range = 0.1,
#     zoom_range = 0.1,
#     horizontal_flip = True,
#     fill_mode = 'nearest'
# )

In [ ]:
batch_size = 64

In [ ]:
# augmented_images = datagen.flow(train_data,train_labels,batch_size=batch_size,shuffle = True)

In [ ]:
# vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# for layer in vgg19_base.layers:
#     layer.trainable = False

In [ ]:
# inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# for layer in inception_base.layers:
#     layer.trainable = False


In [ ]:
 # for layer in inception_base.layers[:249]:  # Freeze until 'mixed9'
 #    layer.trainable = False

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import InceptionV3

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping


### GOOGLENET ARCH

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, AveragePooling2D, GlobalAveragePooling2D, concatenate
# from tensorflow.keras.optimizers import AdamW
# from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping

# # Inception Module
# def inception(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool):
#     # 1x1 Convolution
#     conv1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)

#     # 3x3 Convolution
#     conv3x3_reduce = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
#     conv3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv3x3_reduce)

#     # 5x5 Convolution
#     conv5x5_reduce = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
#     conv5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv5x5_reduce)

#     # Max Pooling
#     pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
#     pool_conv = Conv2D(filters_pool, (1, 1), padding='same', activation='relu')(pool)

#     # Concatenate all branches
#     output = concatenate([conv1x1, conv3x3, conv5x5, pool_conv], axis=-1)
#     return output

# # Custom CNN Model (Updated with Inception Architecture)
# def build_custom_cnn():
#     inp = Input(shape=(224, 224, 3))  # Input shape adjusted to match your original code
#     x = Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(inp)
#     x = MaxPooling2D((3, 3), strides=2)(x)
#     x = Conv2D(64, (1, 1), strides=1, padding='same', activation='relu')(x)
#     x = Conv2D(192, (3, 3), strides=1, padding='same', activation='relu')(x)
#     x = MaxPooling2D((3, 3), strides=2)(x)

#     # Inception Modules
#     x = inception(x, filters_1x1=64, filters_3x3_reduce=96, filters_3x3=128, filters_5x5_reduce=16, filters_5x5=32, filters_pool=32)
#     x = inception(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=192, filters_5x5_reduce=32, filters_5x5=96, filters_pool=64)
#     x = MaxPooling2D((3, 3), strides=2)(x)
#     x = inception(x, filters_1x1=192, filters_3x3_reduce=96, filters_3x3=208, filters_5x5_reduce=16, filters_5x5=48, filters_pool=64)

#     # Auxiliary Output 1
#     aux1 = AveragePooling2D((5, 5), strides=3)(x)
#     aux1 = Conv2D(128, (1, 1), padding='same', activation='relu')(aux1)
#     aux1 = Flatten()(aux1)
#     aux1 = Dense(1024, activation='relu')(aux1)
#     aux1 = Dropout(0.7)(aux1)
#     aux1 = Dense(3, activation='softmax', name='aux1')(aux1)  # Named output layer

#     # Continue with Inception Modules
#     x = inception(x, filters_1x1=160, filters_3x3_reduce=112, filters_3x3=224, filters_5x5_reduce=24, filters_5x5=64, filters_pool=64)
#     x = inception(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=256, filters_5x5_reduce=24, filters_5x5=64, filters_pool=64)
#     x = inception(x, filters_1x1=112, filters_3x3_reduce=144, filters_3x3=288, filters_5x5_reduce=32, filters_5x5=64, filters_pool=64)

#     # Auxiliary Output 2
#     aux2 = AveragePooling2D((5, 5), strides=3)(x)
#     aux2 = Conv2D(128, (1, 1), padding='same', activation='relu')(aux2)
#     aux2 = Flatten()(aux2)
#     aux2 = Dense(1024, activation='relu')(aux2)
#     aux2 = Dropout(0.7)(aux2)
#     aux2 = Dense(3, activation='softmax', name='aux2')(aux2)  # Named output layer

#     # Final Inception Modules
#     x = inception(x, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320, filters_5x5_reduce=32, filters_5x5=128, filters_pool=128)
#     x = MaxPooling2D((3, 3), strides=2)(x)
#     x = inception(x, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320, filters_5x5_reduce=32, filters_5x5=128, filters_pool=128)
#     x = inception(x, filters_1x1=384, filters_3x3_reduce=192, filters_3x3=384, filters_5x5_reduce=48, filters_5x5=128, filters_pool=128)

#     # Final Layers
#     x = GlobalAveragePooling2D()(x)
#     x = Dropout(0.4)(x)
#     out = Dense(3, activation='softmax', name='main_output')(x)  # Named output layer

#     # Create Model
#     model = Model(inputs=inp, outputs=[out, aux1, aux2])
#     return model

# # One-cycle learning rate scheduler function
# def one_cycle_scheduler(epoch, total_epochs, max_lr, min_lr):
#     if epoch < total_epochs / 2:
#         return min_lr + (max_lr - min_lr) * (epoch / (total_epochs / 2))
#     else:
#         return max_lr - (max_lr - min_lr) * ((epoch - total_epochs / 2) / (total_epochs / 2))

# # Compile Model
# model = build_custom_cnn()
# optim = AdamW(learning_rate=0.0001, weight_decay=1e-4)  # AdamW for better optimization
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.1, min_lr=1e-6)
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # Compile with multiple outputs and metrics for each output
# model.compile(optimizer=optim,
#               loss={'main_output': 'categorical_crossentropy', 'aux1': 'categorical_crossentropy', 'aux2': 'categorical_crossentropy'},
#               metrics={'main_output': ['accuracy'], 'aux1': ['accuracy'], 'aux2': ['accuracy']})  # Metrics for each output

# # Learning rate scheduler callback
# epochs = 100
# lr_scheduler = LearningRateScheduler(lambda epoch: one_cycle_scheduler(epoch, epochs, max_lr=0.01, min_lr=0.001))

# # Train the model
# history = model.fit(train_data, {'main_output': train_labels, 'aux1': train_labels, 'aux2': train_labels},
#                     validation_data=(valid_data, {'main_output': valid_labels, 'aux1': valid_labels, 'aux2': valid_labels}),
#                     epochs=epochs,
#                     batch_size=64,
#                     callbacks=[early_stopping, learning_rate_reduction, lr_scheduler])

### VGG19 AND INCEPTION 
VGG19 as a feature extractor and then pass its output to the Inception-based architecture , THEN DENSE LAYERS

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.applications import VGG19
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, AveragePooling2D, GlobalAveragePooling2D, concatenate
# from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping

# # Load VGG19 as a feature extractor
# vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# for layer in vgg19_base.layers:
#     layer.trainable = False  # Freeze VGG19 layers

# # Inception Module
# def inception(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool):
#     # 1x1 Convolution
#     conv1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)

#     # 3x3 Convolution
#     conv3x3_reduce = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
#     conv3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv3x3_reduce)

#     # 5x5 Convolution
#     conv5x5_reduce = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
#     conv5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv5x5_reduce)

#     # Max Pooling
#     pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
#     pool_conv = Conv2D(filters_pool, (1, 1), padding='same', activation='relu')(pool)

#     # Concatenate all branches
#     output = concatenate([conv1x1, conv3x3, conv5x5, pool_conv], axis=-1)
#     return output

# # Build the combined model
# def build_combined_model():
#     # Input layer
#     inp = Input(shape=(224, 224, 3))

#     # VGG19 feature extractor
#     x = vgg19_base(inp)

#     # Inception modules
#     x = inception(x, filters_1x1=64, filters_3x3_reduce=96, filters_3x3=128, filters_5x5_reduce=16, filters_5x5=32, filters_pool=32)
#     x = inception(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=192, filters_5x5_reduce=32, filters_5x5=96, filters_pool=64)
#     x = MaxPooling2D((3, 3), strides=2)(x)
#     x = inception(x, filters_1x1=192, filters_3x3_reduce=96, filters_3x3=208, filters_5x5_reduce=16, filters_5x5=48, filters_pool=64)

#     # Flatten the output
#     x = Flatten()(x)

#     # Fully connected layers from the old architecture
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.2)(x)

#     # Output layer
#     out = Dense(3, activation='softmax')(x)

#     # Create the model
#     model = Model(inputs=inp, outputs=out)
#     return model

# # Learning rate scheduler function
# def one_cycle_scheduler(epoch, total_epochs, max_lr, min_lr):
#     if epoch < total_epochs / 2:
#         return min_lr + (max_lr - min_lr) * (epoch / (total_epochs / 2))
#     else:
#         return max_lr - (max_lr - min_lr) * ((epoch - total_epochs / 2) / (total_epochs / 2))

# # Build the model
# model = build_combined_model()

# # Compile the model
# optim = RMSprop(learning_rate=0.000001)
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.1, min_lr=1e-6)
# model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])

# # Learning rate scheduler callback
# epochs = 100
# lr_scheduler = LearningRateScheduler(lambda epoch: one_cycle_scheduler(epoch, epochs, max_lr=0.01, min_lr=0.001))
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# # Train the model
# history = model.fit(train_data, train_labels,
#                     validation_data=(valid_data, valid_labels),
#                     epochs=epochs,
#                     batch_size=64,
#                     callbacks=[early_stopping,learning_rate_reduction, lr_scheduler])


Phase 1 (Feature Extraction)

VGG19 is frozen (only new layers train).

Uses RMSprop (better for feature extraction).

Trains for 100 epochs.

Phase 2 (Fine-tuning)

Unfreezes top 5 layers of VGG19.

Uses SGD (better for fine-tuning).

Trains for 30 epochs.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, concatenate
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler


# Load VGG19 as base model
vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all VGG19 layers for Phase 1
for layer in vgg19_base.layers:
    layer.trainable = False

# Inception Module Function
def inception_module(x):
    branch1 = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    
    branch2 = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    branch2 = Conv2D(64, (3, 3), padding='same', activation='relu')(branch2)
    
    branch3 = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    branch3 = Conv2D(64, (5, 5), padding='same', activation='relu')(branch3)
    
    branch4 = MaxPooling2D((3, 3), strides=1, padding='same')(x)
    branch4 = Conv2D(64, (1, 1), padding='same', activation='relu')(branch4)

    return concatenate([branch1, branch2, branch3, branch4], axis=-1)

# Build Model
def build_model():
    inp = Input(shape=(224, 224, 3))
    x = vgg19_base(inp)  # Extract features
    x = inception_module(x)
    x = inception_module(x)
    x = GlobalAveragePooling2D()(x)  # Reduce to 2D (batch, 512)
    
    # Fully connected layers
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(3, activation='softmax')(x)  # 3-class classification
    
    model = Model(inputs=inp, outputs=output)
    return model

model = build_model()

# Optimizer for Phase 1
optim1 = RMSprop(learning_rate=0.0001)

# Compile Model for Phase 1
model.compile(optimizer=optim1, loss='categorical_crossentropy', metrics=['accuracy'])

# Learning Rate Scheduler Function
def one_cycle_scheduler(epoch, total_epochs, max_lr, min_lr):
    if epoch < total_epochs / 2:
        return min_lr + (max_lr - min_lr) * (epoch / (total_epochs / 2))
    else:
        return max_lr - (max_lr - min_lr) * ((epoch - total_epochs / 2) / (total_epochs / 2))

# Learning Rate Reduction Callback
lr_scheduler = LearningRateScheduler(lambda epoch: one_cycle_scheduler(epoch, 100, max_lr=0.01, min_lr=0.001))
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.1, min_lr=1e-6)

# Phase 1: Training with EarlyStopping
early_stopping_phase1 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_phase1 = model.fit(train_data, train_labels, 
                           validation_data=(valid_data, valid_labels), 
                           epochs=100, 
                           batch_size=64, 
                           callbacks=[reduce_lr, lr_scheduler, early_stopping_phase1])



In [ ]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import EarlyStopping

# Set up mixed precision policy
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Phase 2: Fine-tuning VGG19 with EarlyStopping
print("\n Training Phase 2: Fine-tuning VGG19 top layers")

# Create the SGD optimizer
optim2 = tf.keras.optimizers.SGD(learning_rate=1e-4, momentum=0.76, nesterov=True, weight_decay=1e-5)

# Wrap the SGD optimizer with LossScaleOptimizer for dynamic loss scaling
loss_scale_optimizer = mixed_precision.LossScaleOptimizer(optim2)

# Unfreeze top layers of VGG19
for layer in vgg19_base.layers[-5:]:
    layer.trainable = True

# Compile the model with the wrapped optimizer
model.compile(optimizer=loss_scale_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping for Phase 2
early_stopping_phase2 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history_phase2 = model.fit(train_data, train_labels, 
                           validation_data=(valid_data, valid_labels), 
                           epochs=30,  # Train for 30 more epochs
                           batch_size=64, 
                           callbacks=[reduce_lr, lr_scheduler, early_stopping_phase2])

In [ ]:
# Learning rate scheduler function
def one_cycle_scheduler(epoch, total_epochs, max_lr, min_lr):
    if epoch < total_epochs / 2:
        return min_lr + (max_lr - min_lr) * (epoch / (total_epochs / 2))
    else:
        return max_lr - (max_lr - min_lr) * ((epoch - total_epochs / 2) / (total_epochs / 2))


In [ ]:
model.summary()

In [ ]:
# Lists to store loss values
train_losses = []
val_losses = []
learning_rates = []

In [ ]:
history= history_phase1

In [ ]:
epochs=100

In [ ]:
# Store the loss values
train_losses = history.history['loss']
val_losses = history.history['val_loss']
learning_rates = [one_cycle_scheduler(epoch, epochs, max_lr=0.01, min_lr=0.001) for epoch in range(epochs)]


In [ ]:
import matplotlib.pyplot as plt

# Get the actual number of epochs trained
actual_epochs = len(history.history['loss'])

# Plotting the loss values
plt.figure(figsize=(12, 6))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(range(1, actual_epochs + 1), history.history['loss'], label='Train Loss')
plt.plot(range(1, actual_epochs + 1), history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot learning rate
plt.subplot(1, 2, 2)
plt.plot(range(1, actual_epochs + 1), history.history['learning_rate'], label='Learning Rate', color='orange')
plt.title('Learning Rate Schedule')
plt.xlabel('Epochs')
plt.ylabel('Learning Rate')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.plot(history.history['loss'],label = 'Training Loss')
plt.plot(history.history['val_loss'],label = 'Validation Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],label = 'Training Accuracy')
plt.plot(history.history['val_accuracy'],label = 'Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
# Identify the final learning rate corresponding to the lowest validation loss
min_val_loss = min(val_losses)
final_lr = learning_rates[val_losses.index(min_val_loss)]
print(f'Final Learning Rate at Lowest Validation Loss: {final_lr}')

## plotting for phase 2

In [ ]:
history= history_phase2
import matplotlib.pyplot as plt
epochs= 30

# Get the actual number of epochs trained
actual_epochs = len(history.history['loss'])

# Plotting the loss values
plt.figure(figsize=(12, 6))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(range(1, actual_epochs + 1), history.history['loss'], label='Train Loss')
plt.plot(range(1, actual_epochs + 1), history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot learning rate
plt.subplot(1, 2, 2)
plt.plot(range(1, actual_epochs + 1), history.history['learning_rate'], label='Learning Rate', color='orange')
plt.title('Learning Rate Schedule')
plt.xlabel('Epochs')
plt.ylabel('Learning Rate')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.plot(history.history['loss'],label = 'Training Loss')
plt.plot(history.history['val_loss'],label = 'Validation Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],label = 'Training Accuracy')
plt.plot(history.history['val_accuracy'],label = 'Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
Normal_test_dir = test_path/"NORMAL"
Bacterial_test_dir = test_path/"BACTERIAL"
Viral_test_dir = test_path/"VIRAL"

In [ ]:
Normal_test_cases = Normal_test_dir.glob("*.jpeg")
Bar_test_cases = Bacterial_test_dir.glob("*.jpeg")
Vir_test_cases = Viral_test_dir.glob("*.jpeg")

In [ ]:
test_data = []

for case in Normal_train_cases:
    test_data.append((case, 0))

for case in Bacterial_train_cases:
    test_data.append((case, 1))
    
for case in Viral_train_cases:
    test_data.append((case,2))


test_data = pd.DataFrame(test_data,columns = ['image','label'],index=None)
test_data = data.sample(frac=1.).reset_index(drop=True)

X_test = test_data['image']
y_test = test_data['label']

In [ ]:
test_data = []
test_labels = []
for img in X_test:
  img = cv2.imread(str(img))
  img = cv2.resize(img, (224,224))
  if img.shape[2] ==1:
    img = np.dstack([img, img, img])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = img.astype(np.float32)/255.
  test_data.append(img)
for label in y_test:
  test_labels.append(label)

test_data = np.array(test_data)
test_labels = np.array(test_labels)
test_labels = to_categorical(test_labels,num_classes = 3)

In [ ]:
test_labels.shape

In [ ]:
# Evaluate the model on the test set
test_results = model.evaluate(test_data, test_labels, batch_size=64)

# Capture the loss and accuracy
test_loss, test_accuracy = test_results[0], test_results[1]

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score

# Get predictions from the model
_y_pred = model.predict(test_data)  # Shape: (num_samples, num_classes)

# Convert softmax probabilities to class labels
y_true = np.argmax(test_labels, axis=1)  # Convert one-hot to class index
y_pred = np.argmax(_y_pred, axis=1)  # Convert probabilities to class index

# Print classification report
print(classification_report(y_true, y_pred))

# Print accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")